In [1]:
import numpy as np
import pandas as pd
import pgaccess as pg

from sklearn.model_selection import train_test_split

In [34]:
# Monthly carrier delay
carrierDelay = pd.read_csv('../data/monthly_carrier_delay.csv')
carrierDelay.set_index(['op_unique_carrier', 'month'], inplace=True)
# Plane Speed
planeSpeed = pd.read_csv('../data/plane_speed.csv')
planeSpeed.set_index(['tail_num'], inplace=True)

# Load the new columns
ratios = pd.read_csv('../data/airport_delays.csv', index_col=0)
dists = pd.read_csv('../data/distance_delays.csv')
months = pd.read_csv('../data/month_delays.csv')
hours = pd.read_csv('../data/hour_delays.csv')

carrierDelay.shape, planeSpeed.shape

((320, 2), (6481, 4))

I'm not including expected flight delay this time, that seems like the least informative column in the previous state, and there might be a more reliable pattern when using city directly

In [20]:
flightData = pg.get_test_data(300000)
flightData['fl_date'] = pd.to_datetime(flightData['fl_date'])
flightData

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,target
0,2018-04-18,DL,DL,DL,1756,DL,N360NB,1756,14869,SLC,...,14747,SEA,"Seattle, WA",1500,1608,N,128.0,1.0,689.0,-21.0
1,2018-04-18,DL,DL,DL,1813,DL,N961AT,1813,10397,ATL,...,15249,TLH,"Tallahassee, FL",1623,1724,N,61.0,1.0,223.0,-10.0
2,2018-04-18,DL,DL,DL,1919,DL,N837DN,1919,12953,LGA,...,13487,MSP,"Minneapolis, MN",605,806,N,181.0,1.0,1020.0,-26.0
3,2018-04-18,DL,DL,DL,2038,DL,N377DA,2038,14747,SEA,...,12892,LAX,"Los Angeles, CA",1900,2140,N,160.0,1.0,954.0,29.0
4,2018-04-18,DL,DL,DL,2117,DL,N347NW,2117,10397,ATL,...,11109,COS,"Colorado Springs, CO",1926,2045,N,199.0,1.0,1184.0,-11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294194,2018-04-18,DL,DL,DL,1528,DL,N889DN,1528,12889,LAS,...,13487,MSP,"Minneapolis, MN",700,1209,N,189.0,1.0,1299.0,-24.0
294195,2018-04-18,DL,DL,DL,1588,DL,N536US,1588,14747,SEA,...,10397,ATL,"Atlanta, GA",2250,634,N,284.0,1.0,2182.0,-14.0
294196,2018-04-18,DL,DL,DL,1639,DL,N937AT,1639,15249,TLH,...,10397,ATL,"Atlanta, GA",1530,1641,N,71.0,1.0,223.0,-17.0
294197,2018-04-18,DL,DL,DL,1674,DL,N3750D,1674,14869,SLC,...,14057,PDX,"Portland, OR",2220,2318,N,118.0,1.0,630.0,-24.0


Link the data columns to our prepared csvs

In [35]:
modelData = flightData[[
    'crs_arr_time', 'crs_elapsed_time', 'distance', 'target',
    'op_unique_carrier', 'fl_date', 'tail_num', 'origin_airport_id',
    'dest_airport_id'
]].copy()
modelData['fl_date'] = modelData.fl_date.dt.month

# Monthly carrier delay
modelData['carrier_delay'] = pd.merge(
    modelData, carrierDelay,
    left_on = ['op_unique_carrier', 'fl_date'],
    right_index = True
)['mean_delay']

# Plane speed
modelData['plane_speed'] = pd.merge(
    modelData, planeSpeed,
    left_on = 'tail_num',
    right_index = True
)['speed']
modelData

,crs_arr_time,crs_elapsed_time,distance,target,op_unique_carrier,fl_date,tail_num,origin_airport_id,dest_airport_id,carrier_delay,plane_speed
0,1608,128.0,689.0,-21.0,DL,4,N360NB,14869,14747,0.163906,0.146002
1,1724,61.0,223.0,-10.0,DL,4,N961AT,10397,15249,0.163906,0.158094
2,806,181.0,1020.0,-26.0,DL,4,N837DN,12953,13487,0.163906,0.137089
3,2140,160.0,954.0,29.0,DL,4,N377DA,14747,12892,0.163906,0.140759
4,2045,199.0,1184.0,-11.0,DL,4,N347NW,10397,11109,0.163906,0.143032
...,...,...,...,...,...,...,...,...,...,...,...
294194,1209,189.0,1299.0,-24.0,DL,4,N889DN,12889,13487,0.163906,0.133691
294195,634,284.0,2182.0,-14.0,DL,4,N536US,14747,10397,0.163906,0.131474
294196,1641,71.0,223.0,-17.0,DL,4,N937AT,15249,10397,0.163906,0.159020
294197,2318,118.0,630.0,-24.0,DL,4,N3750D,14869,14057,0.163906,0.139701


In [36]:
modelData = pd.merge(
    pd.merge(modelData, ratios[['origin_ratio']], left_on='origin_airport_id', right_index=True),
    ratios[['dest_ratio']],
    left_on = 'dest_airport_id',
    right_index = True
)
modelData = pd.merge(
    pd.merge(modelData, dists),
    months,
    left_on = 'fl_date',
    right_on = 'month'
)
modelData

,crs_arr_time,crs_elapsed_time,distance,target,op_unique_carrier,fl_date,tail_num,origin_airport_id,dest_airport_id,carrier_delay,plane_speed,origin_ratio,dest_ratio,distance_ratio,month,month_ratio
0,1608,128.0,689.0,-21.0,DL,4,N360NB,14869,14747,0.163906,0.146002,18835.372340,344620.478723,8105.951643,4.0,44746875.0
1,957,142.0,689.0,14.0,CP,4,N629CZ,14869,14747,1.432123,0.168141,18835.372340,344620.478723,8105.951643,4.0,44746875.0
2,1003,133.0,689.0,-19.0,DL,4,N3745B,14869,14747,0.163906,0.138866,18835.372340,344620.478723,8105.951643,4.0,44746875.0
3,1229,129.0,689.0,11.0,DL,4,N314NB,14869,14747,0.163906,0.144457,18835.372340,344620.478723,8105.951643,4.0,44746875.0
4,1100,135.0,689.0,-13.0,AS,4,N251AK,14869,14747,-1.088908,0.134889,18835.372340,344620.478723,8105.951643,4.0,44746875.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294194,1039,139.0,905.0,-4.0,G4,9,221NV,10466,11980,1.817598,0.135539,15923.138298,2736.968085,516.986981,9.0,18151125.0
294195,2043,187.0,1373.0,-20.0,G4,9,257NV,10466,14512,1.817598,0.134493,15923.138298,-521.010638,109.175449,9.0,18151125.0
294196,1535,20.0,31.0,-20.0,AS,9,N607AS,14256,15841,-1.296606,0.171900,-1002.659574,829.255319,-117.234966,9.0,18151125.0
294197,1550,21.0,31.0,-21.0,AS,9,N607AS,14256,15841,-1.296606,0.171900,-1002.659574,829.255319,-117.234966,9.0,18151125.0


In [37]:
modelData['hour'] = modelData['crs_arr_time'] // 100
modelData = pd.merge(modelData, hours)
modelData

,crs_arr_time,crs_elapsed_time,distance,target,op_unique_carrier,fl_date,tail_num,origin_airport_id,dest_airport_id,carrier_delay,plane_speed,origin_ratio,dest_ratio,distance_ratio,month,month_ratio,hour,hour_ratio
0,1608,128.0,689.0,-21.0,DL,4,N360NB,14869,14747,0.163906,0.146002,1.883537e+04,344620.478723,8105.951643,4.0,4.474688e+07,16,27015148.0
1,1644,132.0,689.0,5.0,OO,4,N184SY,14869,14747,3.523944,0.155048,1.883537e+04,344620.478723,8105.951643,4.0,4.474688e+07,16,27015148.0
2,1643,140.0,689.0,22.0,DL,4,N929DN,10397,12266,0.163906,0.146970,5.761939e+05,538829.255319,8105.951643,4.0,4.474688e+07,16,27015148.0
3,1650,120.0,689.0,-12.0,WN,4,N559WN,13495,13198,4.800241,0.149380,1.625928e+05,97971.010638,8105.951643,4.0,4.474688e+07,16,27015148.0
4,1633,115.0,689.0,-6.0,AS,4,N319AS,14747,14869,-1.088908,0.144348,2.003191e+05,100158.776596,8105.951643,4.0,4.474688e+07,16,27015148.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294194,349,230.0,1598.0,13.0,B6,9,N663JB,12478,14843,2.220871,0.138175,1.840854e+05,128808.244681,2710.539368,9.0,1.815112e+07,3,159160.0
294195,2400,222.0,1249.0,-33.0,B6,1,N640JB,10721,14635,14.760556,0.137931,5.383654e+05,64524.734043,6194.544327,1.0,4.016998e+07,24,16216.0
294196,2400,113.0,621.0,106.0,NK,11,N677NK,13930,10821,1.105945,0.135882,2.238224e+06,130999.734043,6480.099194,11.0,3.189132e+07,24,16216.0
294197,2400,180.0,1197.0,-17.0,B6,2,N318JB,11697,15096,9.964439,0.169510,4.060926e+05,60461.170213,16525.480471,2.0,6.279128e+07,24,16216.0


Make Model

In [38]:
y = modelData['target']
X = modelData[[
    'crs_arr_time', 'crs_elapsed_time', 'distance', 'carrier_delay', 'plane_speed',
    'origin_ratio', 'dest_ratio', 'distance_ratio', 'month_ratio', 'hour_ratio'
]]

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)
Xtrain.shape, Xtest.shape

((220649, 10), (73550, 10))

In [39]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(Xtrain, ytrain)
lr.score(Xtest, ytest)

0.021862365623144364

In [40]:
import statsmodels.api as sm

Xconst = sm.add_constant(Xtrain)
testConst = sm.add_constant(Xtest)

model = sm.OLS(ytrain, Xconst)
res = model.fit()

res.summary()

E:\Users\eboyc\anaconda3\envs\bootcamp\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     492.8
Date:                Wed, 22 Sep 2021   Prob (F-statistic):               0.00
Time:                        17:48:28   Log-Likelihood:            -1.1740e+06
No. Observations:              220649   AIC:                         2.348e+06
Df Residuals:                  220638   BIC:                         2.348e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -6.6885      1.239     -5.400      0.000      -9.116      -4.261
crs_arr_time         0.0001      0.000      0.272      0.785      -0.001       0.001
crs_elapsed_time    -0.0673      0.008     -8.404      0.000      -0.083      -0.052
distance             0.0073      0.001      7.230      0.000       0.005       0.009
carrier_delay        0.9023      0.026     35.085      0.000       0.852       0.953
plane_speed          6.5742      6.333      1.038      0.299      -5.838      18.987
origin_ratio      2.706e-06    2.1e-07     12.891      0.000    2.29e-06    3.12e-06
dest_ratio        4.283e-06    2.4e-07     17.879      0.000    3.81e-06    4.75e-06
distance_ratio       0.0001   1.43e-05      7.159      0.000    7.44e-05       0.000
month_ratio       9.285e-09   3.98e-09      2.333      0.020    1.48e-09    1.71e-08
hour_ratio        2.674e-07    1.3e-08     20.583      0.000    2.42e-07    2.93e-07
==============================================================================
Omnibus:                   327597.908   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        224718962.839
Skew:                           8.970   Prob(JB):                         0.00
Kurtosis:                     158.309   Cond. No.                     4.29e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.29e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""